In [34]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold  
from sklearn.feature_selection import SelectPercentile, f_regression
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
import copy as cp
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator

In [35]:
train = pd.read_csv("../data/processed/train.csv")

In [36]:
train.pop("id")
target = train.pop("血糖")

In [37]:
X = train.as_matrix()
Y = target.as_matrix()

In [38]:
def get_dummies(train, test):
    df_train = pd.DataFrame(train)
    df_test = pd.DataFrame(test)
    
    df_total = pd.concat([df_train, df_test])

    columns = df_total.columns.tolist()
    one_hots = pd.DataFrame()
    for column in columns:
        one_hot = pd.get_dummies(df_total[column], prefix=column)
        one_hots = pd.concat([one_hot,one_hots],axis=1)
        df_total.pop(column)
    df_total = one_hots
    
    length = train.shape[0]
    
    train = df_total[0:length]
    test = df_total[length:]

    
    return train.as_matrix(), test.as_matrix()
    

In [42]:
N = 5
kf = KFold(n_splits=N, random_state=42)
result_mean = 0.0

for train_index, test_index in kf.split(X):
    training_features, training_target = X[train_index], Y[train_index]
    testing_features, testing_target = X[test_index], Y[test_index]

    exported_pipeline = make_pipeline(
        MaxAbsScaler(),
        StackingEstimator(estimator=LinearSVR(C=0.01, dual=False, epsilon=1.0, loss="squared_epsilon_insensitive", tol=0.001)),
        StackingEstimator(estimator=RidgeCV()),
        lgb.LGBMRegressor(objective='regression',
                        boosting_type ="GBDT",
                        num_leaves=17,
                        learning_rate=0.01,
                        feature_fraction=0.5,
                        bagging_fraction=0.5,
                        bagging_freq=5,
                        reg_alpha=1,
                        reg_lambda=0.5,
                        n_estimators=500)
    )
    
    exported_pipeline.fit(training_features, training_target)
    results = exported_pipeline.predict(testing_features)

    result_mean += np.round(mean_squared_error(testing_target, results), 5)
result_mean /= N
print("Mean squared error: %.5f" % (result_mean/2))

C:\Users\altman\Anaconda3\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\altman\Anaconda3\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)
C:\Users\altman\Anaconda3\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\altman\Anaconda3\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWar

Mean squared error: 0.95963


In [33]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mse'},
    'num_leaves': 17,
    'learning_rate': 0.01,
    'lambda_l1': 1,
    'lambda_l2': 0.5,
    'cat_smooth': 10,
    'feature_fraction': 0.5,
    'bagging_fraction':0.5,
    'bagging_freq': 5,
    'verbose': 0
}

N = 5
kf = KFold(n_splits=N, random_state=42)
result_mean = 0.0

for train_index, test_index in kf.split(X):
    training_features, training_target = X[train_index], Y[train_index]
    testing_features, testing_target = X[test_index], Y[test_index]

    lgb_train = lgb.Dataset(training_features, training_target)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500)
    results = gbm.predict(testing_features)

    result_mean += np.round(mean_squared_error(testing_target, results), 5)
result_mean /= N
print("Mean squared error: %.5f" % (result_mean/2))

Mean squared error: 0.97715


In [ ]:
N = 5
kf = KFold(n_splits=N, random_state=42)
result_mean = 0.0

for train_index, test_index in kf.split(X):
    training_features, training_target = X[train_index], Y[train_index]
    testing_features, testing_target = X[test_index], Y[test_index]

    exported_pipeline = make_pipeline(
        MaxAbsScaler(),
        StackingEstimator(estimator=LinearSVR(C=0.01, dual=False, epsilon=1.0, loss="squared_epsilon_insensitive", tol=0.001)),
        StackingEstimator(estimator=RidgeCV()),
        ExtraTreesRegressor(bootstrap=False, min_samples_leaf=4, min_samples_split=6, n_estimators=100)
    )
    exported_pipeline.fit(training_features, training_target)
    results = exported_pipeline.predict(testing_features)

    result_mean += np.round(mean_squared_error(testing_target, results), 5)
result_mean /= N
print("Mean squared error: %.5f" % (result_mean/2))